In [13]:
import warnings

import pandas as pd
import numpy as np
import torch

from MLWrappers import PyTorchBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [14]:
# Definition of the PyTorch model

class Net(torch.nn.Module):
    """PyTorch neural network."""

    def __init__(self, n_features, n_classes):
        super().__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.fc1 = torch.nn.Linear(self.n_features, 64)
        self.fc99 = torch.nn.Linear(64, self.n_classes)
        self.dropout = torch.nn.Dropout(0.30)

    def forward(self, x):
        x = torch.nn.functional.tanh(self.fc1(x))
        x = self.dropout(x)
        out = torch.nn.functional.softmax(self.fc99(x), dim=1)
        return out

In [15]:
MODEL_NAME = 'nn_torch'
DS_NAME = 'adult'
DATA_FOLDER = f'./data/{DS_NAME}'

# We load the data used to train and test the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
train_label = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_label.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)


In [16]:
# we load the target black box model using our wrapper

# For PyTorch we can load directly the jitted model
target = PyTorchBlackBox(f'./models/{DS_NAME}_{MODEL_NAME}.pt')

# Otherwise we can load the PyTorch state discitonary
target = PyTorchBlackBox(f'./models/{DS_NAME}_{MODEL_NAME}.pkl',
                         nn_class=Net(train_set.shape[1], len(np.unique(train_label))))

In [17]:
# We can define the parameters to be passed to the shadow models (in our case random forests)
shadow_forest_params = {'n_estimators': 100,
                        'max_depth': 10}

In [18]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target,
                       n_shadow_models=5)
label_only = LabelOnlyPrivacyAttack(target,
                                    n_shadow_models=1,
                                    shadow_model_type='rf',
                                    shadow_model_params=shadow_forest_params,
                                    n_noise_samples_fit=1000)
aloa = AloaPrivacyAttack(target,
                         n_shadow_models=1,
                         shadow_model_type='rf',
                         shadow_model_params=shadow_forest_params,
                         n_noise_samples_fit=1000,
                         shadow_test_size=0.3,
                         undersample_attack_dataset=True)
attacks = [mia, label_only, aloa]

In [19]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the list of attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [20]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set, save_folder = f'./results_{DS_NAME}_{MODEL_NAME}')

100%|██████████| 3622/3622 [00:17<00:00, 203.67it/s]


In [21]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)

100%|██████████| 24134/24134 [02:03<00:00, 194.77it/s]


{'mia_attack': {'classification_report': {'IN': {'precision': 0.7999576540334533, 'recall': 0.9784534106800642, 'f1-score': 0.88024789152416, 'support': 19307}, 'OUT': {'precision': 0.19845857418111754, 'recall': 0.021338305365651542, 'f1-score': 0.03853348297792742, 'support': 4827}, 'accuracy': 0.7870224579431507, 'macro avg': {'precision': 0.49920811410728544, 'recall': 0.49989585802285785, 'f1-score': 0.45939068725104376, 'support': 24134}, 'weighted avg': {'precision': 0.6796528534016797, 'recall': 0.7870224579431507, 'f1-score': 0.7118980344738299, 'support': 24134}}}, 'label_only_attack': {'classification_report': {'IN': {'precision': 0.8005506714327119, 'recall': 0.8583933288444605, 'f1-score': 0.8284635956909695, 'support': 19307}, 'OUT': {'precision': 0.20337995337995338, 'recall': 0.14460327325460948, 'f1-score': 0.16902772732776367, 'support': 4827}, 'accuracy': 0.7156294025026932, 'macro avg': {'precision': 0.5019653124063326, 'recall': 0.501498301049535, 'f1-score': 0.498

In [22]:
results['mia_attack']['classification_report']

{'IN': {'precision': 0.7999576540334533,
  'recall': 0.9784534106800642,
  'f1-score': 0.88024789152416,
  'support': 19307},
 'OUT': {'precision': 0.19845857418111754,
  'recall': 0.021338305365651542,
  'f1-score': 0.03853348297792742,
  'support': 4827},
 'accuracy': 0.7870224579431507,
 'macro avg': {'precision': 0.49920811410728544,
  'recall': 0.49989585802285785,
  'f1-score': 0.45939068725104376,
  'support': 24134},
 'weighted avg': {'precision': 0.6796528534016797,
  'recall': 0.7870224579431507,
  'f1-score': 0.7118980344738299,
  'support': 24134}}

In [23]:
results['label_only_attack']['classification_report']

{'IN': {'precision': 0.8005506714327119,
  'recall': 0.8583933288444605,
  'f1-score': 0.8284635956909695,
  'support': 19307},
 'OUT': {'precision': 0.20337995337995338,
  'recall': 0.14460327325460948,
  'f1-score': 0.16902772732776367,
  'support': 4827},
 'accuracy': 0.7156294025026932,
 'macro avg': {'precision': 0.5019653124063326,
  'recall': 0.501498301049535,
  'f1-score': 0.4987456615093666,
  'support': 24134},
 'weighted avg': {'precision': 0.6811115790302644,
  'recall': 0.7156294025026932,
  'f1-score': 0.6965709572311536,
  'support': 24134}}

In [24]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.7999585578118524,
  'recall': 0.9997928212565391,
  'f1-score': 0.8887814535994659,
  'support': 19307},
 'OUT': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4827},
 'accuracy': 0.7998259716582414,
 'macro avg': {'precision': 0.3999792789059262,
  'recall': 0.49989641062826956,
  'f1-score': 0.44439072679973296,
  'support': 24134},
 'weighted avg': {'precision': 0.6399602169418014,
  'recall': 0.7998259716582414,
  'f1-score': 0.7110177974908796,
  'support': 24134}}